**Imports**

In [ ]:
# wiesel : airsense_env_tsai_031
from imports import * 
from functions import *
from archs import *
from mylearner import *
%matplotlib inline
# %load_ext tensorboard

**Loading annotated data**

In [ ]:
from loading_data import *

**Encode target classes (person and window open) classes**

In [ ]:
df1 = encode_classes(d1)
df2 = encode_classes(d2)
df = encode_classes(data)

**filtering data**

In [ ]:
# h2s column has only NaN values, also drop timestamp,datetime,deviceid
df.drop(columns=['h2s','timestamp','deviceid'],inplace=True)
df1.drop(columns=['h2s','timestamp','deviceid'],inplace=True)
df2.drop(columns=['h2s','timestamp','deviceid'],inplace=True)

**Missing values, drop NAN**

In [ ]:
plot_missing(df1)
plot_missing(df2)

#### substitute NaN values with mean 
## polynomial interpolation with degree > 1 uses index, also convert dtype to float to work
df1 = impute_NaN(df1)
df2 = impute_NaN(df2)
############## conmbine into df 

df = concat(df1,df2)
df.columns=df1.columns

### drop datetime column and store it before for visualization
datetime_df = df.datetime
datetime_df1 = df1.datetime
datetime_df2 = df2.datetime

df.drop(columns=['datetime'],inplace=True)
df1.drop(columns=['datetime'],inplace=True)
df2.drop(columns=['datetime'],inplace=True)



**minimized dataset for our classification task, drop  rows for testing**

In [ ]:

print(df['person'].value_counts(),'\n',df['window_open'].value_counts(),'\n')

#### combine classes of more than 1 person to 1 person as there exists too little of them
lc = df.loc[lambda x: x['person'] > 1]
df.loc[lc.index,'person'] = 1

lc = df1.loc[lambda x: x['person'] > 1]
df1.loc[lc.index,'person'] = 1

lc = df2.loc[lambda x: x['person'] > 1]
df2.loc[lc.index,'person'] = 1

print(df['person'].value_counts(),'\n',df['window_open'].value_counts(),'\n')


############### drop  rows for testing
### november (one month) for testing
df2_test = df2.iloc[df2.shape[0]-22000:df2.shape[0]-21700] ## to get better visualization on less focused data
datetime_test = datetime_df2.iloc[datetime_df2.shape[0]-22000:datetime_df2.shape[0]-21700]
df2 = df2.drop(index = df2.index[df2.shape[0]-22000:]) ### to keep the sequence order drop from last
datetime_df2 = datetime_df2.drop(index = datetime_df2.index[datetime_df2.shape[0]-22000:])


### for visualization
datetime_train = concat(datetime_df1,datetime_df2)
df_train = concat(df1,df2)
df_train.columns=df1.columns

# df2.reset_index(drop=True,inplace=True)
# df2_test.reset_index(drop=True,inplace=True)

df2_test_targets = df2_test.filter(['person','window_open'])
df2_test_features = df2_test.drop(columns=['person','window_open']) 




targets = df.filter(['person','window_open'])
features = df.drop(columns=['person','window_open']) 


targets1 = df1.filter(['person','window_open'])
features1 = df1.drop(columns=['person','window_open']) 

targets2 = df2.filter(['person','window_open'])
features2 = df2.drop(columns=['person','window_open']) 



### minimized data
l_ = ['humidity','temperature','tvoc','oxygen','co2','co','pressure','o3','sound','person','window_open']
f_ = ['humidity','temperature','tvoc','oxygen','co2','co','pressure','o3','sound']
### from leipzig data
# l_ = ['humidity_abs','temperature','tvoc','oxygen','co2','co','no2','o3','person','window_open']
# f_ = ['humidity_abs','temperature','tvoc','oxygen','co2','co','no2','o3']

df_mini1 = df1.filter(l_)
print(df_mini1.shape,'\n',df_mini1['person'].value_counts(),'\n',df_mini1['window_open'].value_counts(),'\n')
features_mini1 = features1.filter(f_)
# print(targets1.iloc[:,0].value_counts())

df_mini2 = df2.filter(l_)
print(df_mini2.shape,'\n',df_mini2['person'].value_counts(),'\n',df_mini2['window_open'].value_counts(),'\n')
features_mini2 = features2.filter(f_)
# print(targets2.iloc[:,0].value_counts())

df2_test_mini = df2_test.filter(l_)
print(df2_test_mini.shape,'\n',df2_test_mini['person'].value_counts(),'\n',df2_test_mini['window_open'].value_counts(),'\n')
df2_test_features_mini = df2_test_features.filter(f_)

## whole df
df_mini = df.filter(l_)
print(df_mini.shape,'\n',df_mini['person'].value_counts(),'\n',df_mini['window_open'].value_counts(),'\n')
features_mini = features.filter(f_)
# print(targets2.iloc[:,0].value_counts())


**Perform undersampling and segmentation**

read yaml config file

In [ ]:
with open('config_LSTM.yaml') as f:
    hyperparams = yaml.load(f,SafeLoader)
    
under_window = hyperparams['sample_segment']['under_window']
seq_len = hyperparams['sample_segment']['seq_len']
stride = hyperparams['sample_segment']['stride']
sliding_mode = hyperparams['sample_segment']['sliding_mode']
all_features = hyperparams['sample_segment']['all_features']

under sample and segment

In [ ]:
## undersampling and sliding
sampling = hyperparams['sample_segment']['sampling']
if sampling:
    if all_features:
        X1,y1= under_sample(df1,under_window,seq_len,stride,sliding_mode=sliding_mode)
        X2,y2= under_sample(df2,under_window,seq_len,stride,sliding_mode=sliding_mode)
    else:
        X1,y1= under_sample(df_mini1,under_window,seq_len,stride,sliding_mode=sliding_mode)
        X2,y2= under_sample(df_mini2,under_window,seq_len,stride,sliding_mode=sliding_mode)
else:
    if all_features:
        X1,y1= sliding(seq_len,stride,features1,targets1,mode=sliding_mode)
        X2,y2= sliding(seq_len,stride,features2,targets2,mode=sliding_mode)
    else:
        X1,y1= sliding(seq_len,stride,features_mini1,targets1,mode=sliding_mode)
        X2,y2= sliding(seq_len,stride,features_mini2,targets2,mode=sliding_mode)



X = concat(X1,X2)
y = concat(y1,y2)
X.shape, y.shape


**splitting sets and standardization**

**using sklearn**

In [ ]:
###### splitting
# splits = TrainValidTestSplitter(valid_size=0.2,test_size=0.1)(y) ##### we have test set here
splits = TrainValidTestSplitter(valid_size=0.1)(y) ##### we DON'T have test set here
x_train = np.zeros(X[splits[0]].shape,dtype=np.float32)
y_train = y[splits[0]]
x_valid = np.zeros(X[splits[1]].shape,dtype=np.float32)
y_valid = y[splits[1]]
#### if splitted
# x_test = np.zeros(X[splits[2]].shape,dtype=np.float32)
# y_test = y[splits[2]]

#different test set
### uses stride = 1 to check all time points
if all_features:
    x_test_, y_test = sliding(seq_len,1,df2_test_features,df2_test_targets,mode=sliding_mode)
else:
    x_test_, y_test = sliding(seq_len,1,df2_test_features_mini,df2_test_targets,mode=sliding_mode)
x_test = np.zeros(x_test_.shape,dtype=np.float32) 
#################



#### standardization
scalers = {}
for i in range(x_train.shape[1]): ## n_features
    scalers[i] = StandardScaler()
    scalers[i].fit(np.unique((X[splits[0]])[:, i, :]).reshape(-1,1)) ### as we have overlapping samples
    x_train[:, i, :] = scalers[i].transform((X[splits[0]])[:, i, :].reshape(-1,1)).reshape(x_train.shape[0],x_train.shape[-1])
    x_valid[:, i, :] = scalers[i].transform((X[splits[1]])[:, i, :].reshape(-1,1)).reshape(x_valid.shape[0],x_valid.shape[-1])
    # x_test[:, i, :] = scalers[i].transform((X[splits[2]])[:, i, :].reshape(-1,1)).reshape(x_test.shape[0],x_test.shape[-1]) ## from splitting
    x_test[:, i, :] = scalers[i].transform((x_test_)[:, i, :].reshape(-1,1)).reshape(x_test.shape[0],x_test.shape[-1]) ## test set from outside
print(x_train.shape,x_valid.shape,x_test.shape)

**tsai LEARNER**

read config file

In [ ]:
with open('config_LSTM.yaml') as f:
    hyperparams = yaml.load(f,SafeLoader)

epochs = hyperparams['model']['epochs']
bs = hyperparams['model']['bs']
num_workers = hyperparams['model']['num_workers']
embed_size = hyperparams['model']['embed_size']

**Train (X_train, X_validation)

In [ ]:
######## remove sigmoid from the network*****************

# tfms  = [None, Categorize()] ## single label
tfms  = [None, MultiCategorize()] ## multi label
batch_tfms = TSStandardize()

## single label
# tsets = TSDatasets(x_train, y_train[:,0], tfms=tfms, inplace=True)
# vsets = TSDatasets(x_valid, y_valid[:,0], tfms=tfms, inplace=True)
## multi label
tsets = TSDatasets(x_train, y_train, tfms=tfms, inplace=True)
vsets = TSDatasets(x_valid, y_valid, tfms=tfms, inplace=True)

######################
dls   = TSDataLoaders.from_dsets(tsets, vsets, bs = bs, num_workers=num_workers)#,batch_tfms=batch_tfms) ### note the normalization


#############################################################
k = {'hidden_size':22,'rnn_dropout':0.4}
network = 'LSTM'

model = create_model(MyLSTM, dls.vars,dls.c,dls,**k)
# learn = Learner(dls, model, metrics=accuracy,cbs = ShowGraph()) ### single label
#### note the accuracy_multi and MultiCategorize() contains BCEwithlogits not normal BCE
learn = Learner(dls, model, metrics=F1_multi,cbs = ShowGraph(),lr = 1e-4 ) ### multi label


###########################################################################
start = time.time()
learn.fit_one_cycle(epochs,learn.lr_find().valley,cbs = EarlyStoppingCallback(monitor='valid_loss', min_delta=0.00001, patience=5)) ## learning rate annealing
elapsed = time.time()-start
print(f'time= {elapsed}')
print(f'No. of trainable parameters= {count_parameters(model)}')
print(f'train loss = {learn.recorder.values[-1][0]}, valid loss = {learn.recorder.values[-1][1]}')
plt.figure()
plt.plot(learn.recorder.lrs)

print(learn.loss_func)
metrics = np.array(learn.recorder.values)
print('*** metrics: ' , learn.recorder.values[-1])

# save the model as state dictionary 
torch.save(learn.model.state_dict(), f'{learn.model._get_name()}.pt')


### To read already available networks

In [ ]:
# tfms  = [None, MultiCategorize()] ## multi label
# batch_tfms = TSStandardize()

# ## single label
# # tsets = TSDatasets(x_train, y_train[:,0], tfms=tfms, inplace=True)
# # vsets = TSDatasets(x_valid, y_valid[:,0], tfms=tfms, inplace=True)
# ## multi label
# tsets = TSDatasets(x_train, y_train, tfms=tfms, inplace=True)
# vsets = TSDatasets(x_valid, y_valid, tfms=tfms, inplace=True)
# ######################
# dls   = TSDataLoaders.from_dsets(tsets, vsets, bs = bs, num_workers=num_workers)#,batch_tfms=batch_tfms) ### note the normalization


# k = {'hidden_size':22,'rnn_dropout':0.4}
# network = 'LSTM'


# # load 
# pretrained_dict = torch.load(f'/afs/tu-chemnitz.de/home/urz/a/abom/internship/models/{MyLSTM.__name__}.pt')
# model = create_model(MyLSTM, dls.vars,dls.c,dls,**k)
# model.load_state_dict(pretrained_dict)

# # learn = Learner(dls, model, metrics=accuracy,cbs = ShowGraph()) ### single label
# #### note the accuracy_multi and MultiCategorize() contains BCEwithlogits not normal BCE
# learn = Learner(dls, model, metrics=accuracy_multi,cbs = ShowGraph()) ### multi label

### EVALUATION ON TEST DATA SET + Using PR curves**

In [ ]:
before = learn.recorder.values
print('before:', before)
learn.plot_metrics()

In [ ]:
#### Adding dataset to apply same dls valid tfms on test
def evaluate_test_data(_x_test, _y_test, _tfms,_learn):
    x_test = _x_test
    y_test = _y_test
    tfms = _tfms
    learn = _learn
    ### standardize test set with train parameters (apply Multicategorize transform)
    test_set = TSDatasets(x_test, y_test, tfms=tfms, inplace=True) # multi_label
    # test_set = TSDatasets(x_test, y_test[:,0], tfms=tfms, inplace=True) # single_label
    test_dl = TSDataLoader(test_set, bs = bs, num_workers=num_workers)#,batch_tfms=batch_tfms) ### note the normalization
    # next(iter(test_dl))
    

    #### evaluating

    test_probas, test_targets, test_preds = learn.get_preds(dl=test_dl,with_decoded=True)
    test_preds = test_preds.long() ### to convert bool to int

    ###############################################################################################
    ###### use PR curve to get best threshold for each class on given test set
    ##############################################################################################
    # # # class person
    # # threshold_person, _ = plot_PR_curve('person',test_targets[:,0],test_probas[:,0])
    # # ### class window_open
    # # threshold_window, _ =plot_PR_curve('window',test_targets[:,1],test_probas[:,1])
    # threshold_person,threshold_window = plot_PR_curve_both(test_targets,test_probas)
    # test_preds[:,0] = (test_probas[:,0]>=threshold_person)
    # test_preds[:,1] = (test_probas[:,1]>=threshold_window)
    #################################################################################################

    ## single label
    ### number of classes in validation set
    # n_classes = pd.DataFrame(test_targets).value_counts().shape[0]
    # plot_confusion(test_targets,test_preds,n_classes,name = 'person')
    # print(f'Accuracy: {(test_targets == test_preds).float().mean():0.5f}')  
    # print(f'F1 score: {f1_score(test_targets,test_preds)}\n') # to get f1 score for each class use (average=None)
    # print(f'precision: {precision_score(test_targets,test_preds)}\n') 
    # print(f'recall: {recall_score(test_targets,test_preds)}\n') 

    ## multi label
    print(f'Accuracy [Both]: {(100/test_targets.shape[0])*torch.all((test_preds == test_targets),dim=1).sum():0.5f}') ## multi-label
    print(f'Accuracy [person]: {(100/test_targets.shape[0])*(test_preds[:,0] == test_targets[:,0]).sum():0.5f}') ## multi-label
    print(f'Accuracy [window_open]: {(100/test_targets.shape[0])*(test_preds[:,1] == test_targets[:,1]).sum():0.5f}') ## multi-label
    n_classes = test_targets.shape[1]

    ## confusion matrices
    plot_confusion_both(test_targets,test_preds,n_classes)
    ## person class
    # plot_confusion(test_targets[:,0],test_preds[:,0],n_classes,name = 'person')
    print(f'F1 score [person]: {f1_score(test_targets[:,0],test_preds[:,0])}\n')
    print(f'precision [person]: {precision_score(test_targets[:,0],test_preds[:,0])}\n') 
    print(f'recall [person]: {recall_score(test_targets[:,0],test_preds[:,0])}\n') 

    ### window class
    # plot_confusion(test_targets[:,1],test_preds[:,1],n_classes,name = 'window')
    print(f'F1 score [window]: {f1_score(test_targets[:,1],test_preds[:,1])}\n') 
    print(f'precision [window]: {precision_score(test_targets[:,1],test_preds[:,1])}\n') 
    print(f'recall [window]: {recall_score(test_targets[:,1],test_preds[:,1])}\n') 
    
    return test_targets,test_preds


test_targets,test_preds = evaluate_test_data(_x_test= x_test, _y_test = y_test, _tfms = tfms,_learn = learn)


**investigate distribution of wrong predictions, and smoothing** 

In [ ]:
#### single label
# ## plot targets vs predictions person
# plot_distribution(test_targets,test_preds,'person')
# ## plot FP, FN 
# plot_fp_fn(test_targets,test_preds,'person')
#############################################################################################################
### smoothing
# test_preds = smoothing_predictions(test_preds,2)
###############################################################################################################
#### multi label

fig = plot_distribution_both(test_targets,test_preds, network)


**optuna for HP optimization**

load yaml file

In [ ]:
with open('config_LSTM.yaml') as f:
    hyperparams = yaml.load(f,SafeLoader)
epochs = hyperparams['optuna']['epochs']
optuna_trials = hyperparams['optuna']['trials']
bs = hyperparams['model']['bs']
num_workers = hyperparams['model']['num_workers']

optimize (notice freeze for autoencoder)

In [ ]:
tfms  = [None, MultiCategorize()] 
batch_tfms = TSStandardize()


tsets = TSDatasets(x_train, y_train, tfms=tfms, inplace=True)
# vsets = TSDatasets(x_test, y_test, tfms=tfms, inplace=True)
vsets = TSDatasets(x_valid, y_valid, tfms=tfms, inplace=True)

dls   = TSDataLoaders.from_dsets(tsets, vsets, bs = bs, num_workers=num_workers)#,batch_tfms=batch_tfms) ### note the normalization


def objective(trial:optuna.Trial):

    rt = trial.study.study_name.replace('_LSTM_study','')
    study_folder = f'/scratch/aifa/MyRepo/SmartAirsense/internship/TimeSeriesClassification/Experiments/LSTM/{rt}'
    dat_ = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    trial_folder = os.path.join(study_folder,f'trial_{trial._trial_id}_{dat_}')
    if not os.path.exists(trial_folder):
        os.mkdir(trial_folder)
    
    # Define search space here. Supported search space are 

    rnn_dropout = trial.suggest_float("rnn_dropout", 0.1, 0.5, step = 0.1)
    hidden_size = trial.suggest_int('hidden_size',10,80,step = 2)
    lr = trial.suggest_float("Learning_rate", 1e-8, 1e-4, log=True)

    k = {'hidden_size':hidden_size,'rnn_dropout':rnn_dropout}

    model = create_model(MyLSTM, dls.vars,dls.c,dls,**k)
    #### note the F1_multi and MultiCategorize() contains BCEwithlogits not normal BCE
    learn = Learner(dls, model, metrics=F1_multi, lr=lr)
 
    ######################################
    learn.fit_one_cycle(epochs,learn.lr_find().valley,cbs = EarlyStoppingCallback(monitor='valid_loss', min_delta=0.00001, patience=5)) ## learning rate annealing
    
    # # return the F1 score - learn.recorder.values returns [[train_loss, valid_loss, F1_multi]]
    metric_before_pred = learn.recorder.values[-1][-1]
    
    print('****** before_plot: ', learn.recorder.values[-1])
    test_targets,test_preds = evaluate_test_data(_x_test= x_test, _y_test = y_test, _tfms = tfms,_learn = learn)
    fig = plot_distribution_both(test_targets,test_preds, network='LSTM')
    fig.savefig(os.path.join(trial_folder, 'plot_dist.png'))
    print('****** after_plot: ', learn.recorder.values[-1])
    
    trial_params = os.path.join(trial_folder,'trial_params.yaml')
    with open(trial_params,'w') as f:
        for key, value in trial.params.items():
            f.write("    {}: {}\n".format(key, value))
    
    return metric_before_pred
    # # return the F1 score - learn.recorder.values returns [[train_loss, valid_loss, F1_multi]]
    # return learn.recorder.values[-1][-1]

    # # return the valid_loss - learn.recorder.values returns [[train_loss, valid_loss, acc]]
    # return learn.recorder.values[-1][1]

In [ ]:
dat = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
lstm_folder = f'/scratch/aifa/MyRepo/SmartAirsense/internship/TimeSeriesClassification/Experiments/LSTM/{dat}'
if not os.path.exists(lstm_folder):
    os.mkdir(lstm_folder)

study = optuna.create_study(direction='maximize', 
                            pruner=optuna.pruners.HyperbandPruner(),
                            storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
                            study_name=f"{dat}_LSTM_study")

study.optimize(objective, n_trials=optuna_trials, show_progress_bar=True)

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
best_trial_params = os.path.join(lstm_folder,'best_trial_params.yaml')
with open(best_trial_params,'w') as f:
    for key, value in trial.params.items():
        f.write("    {}: {}\n".format(key, value))

In [ ]:


optimization_history = optuna.visualization.plot_optimization_history(study)
param_importances = optuna.visualization.plot_param_importances(study)
slice = optuna.visualization.plot_slice(study)
parallel_coordinate = optuna.visualization.plot_parallel_coordinate(study)

optimization_history.write_image(os.path.join(lstm_folder,'optimization_history.png'))
param_importances.write_image(os.path.join(lstm_folder,'param_importances.png'))
slice.write_image(os.path.join(lstm_folder,'slice.png'))
parallel_coordinate.write_image(os.path.join(lstm_folder,'parallel_coordinate.png'))

display(optimization_history)
display(param_importances)
display(slice)
display(parallel_coordinate)    
